<a href="https://colab.research.google.com/github/vitroid/PythonTutorials/blob/master/2%20Advanced/065Voronoi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 065 Voronoi

[Voronoiの紹介スライド](https://github.com/vitroid/PythonTutorials/blob/master/2%20Advanced/Voronoi.pdf?raw=true) (In Japanese)


In [ ]:
# Locations of the prefectural capitals of Japan
# https://www.benricho.org/chimei/latlng_data.html

import numpy as np
pos = np.fromstring("""
43.06417	141.34694
40.82444	140.74
39.70361	141.1525
38.26889	140.87194
39.71861	140.1025
38.24056	140.36333
37.75	140.46778
36.34139	140.44667
36.56583	139.88361
36.39111	139.06083
35.85694	139.64889
35.60472	140.12333
35.68944	139.69167
35.44778	139.6425
37.90222	139.02361
36.69528	137.21139
36.59444	136.62556
36.06528	136.22194
35.66389	138.56833
36.65139	138.18111
35.39111	136.72222
34.97694	138.38306
35.18028	136.90667
34.73028	136.50861
35.00444	135.86833
35.02139	135.75556
34.68639	135.52
34.69139	135.18306
34.68528	135.83278
34.22611	135.1675
35.50361	134.23833
35.47222	133.05056
34.66167	133.935
34.39639	132.45944
34.18583	131.47139
34.06583	134.55944
34.34028	134.04333
33.84167	132.76611
33.55972	133.53111
33.60639	130.41806
33.24944	130.29889
32.74472	129.87361
32.78972	130.74167
33.23806	131.6125
31.91111	131.42389
31.56028	130.55806
26.2125	127.68111
""", sep=" ").reshape(-1,2)[:,::-1]
# Split the data by spaces,
# read as an array of numbers,
# convert it to a 2D-array,
# and exchange the two columns.

pos

In [ ]:
from matplotlib import pyplot as plt
fig = plt.figure()
ax = plt.axes()

plt.scatter(pos[:,0], pos[:,1])
plt.axis("square")
plt.show()

県庁所在地をつないだDelaunay図

Delaunay diagram by connecting nearby capitals.

In [ ]:
# Delaunay triangulation
from scipy.spatial import Delaunay
tri = Delaunay(pos)
plt.triplot(pos[:,0], pos[:,1], tri.simplices)
plt.axis("square")
plt.show()

2次元のボロノイ図を描くのはライブラリにまかせる。

The smart library can even make the Voronoi diagram from the locations of the prefectural capitals.

In [ ]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.voronoi_plot_2d.html
from scipy.spatial import Voronoi, voronoi_plot_2d
vor = Voronoi(pos)
fig = voronoi_plot_2d(vor, show_vertices=False, line_colors='orange',
                line_width=2, line_alpha=0.6, point_size=2)
plt.triplot(pos[:,0], pos[:,1], tri.simplices, c='red')
#plt.xlim(127.5,145)
plt.ylim(25, 45)
plt.axis([127.5, 145, 25, 45])
plt.gca().set_aspect('equal', adjustable='box')
fig.set_size_inches(11,11)
plt.show()

歪んで見えるのは、縦と横の目盛が違うから。

境界線をより正確に描くためには、球面幾何学に準じた非ユークリッドVoronoi分割を適用する必要がある。(二等分線が大円になる)

In order to draw the boundary more accurately, it is necessary to apply a non-Euclidean Voronoi division according to spherical geometry. (The bisectors become geodesics.)


もうちょっと少ない点で。例えば4点の場合。

Another example with fewer points.

In [ ]:
from scipy.spatial import Voronoi, voronoi_plot_2d
points = np.array([[0, -0.1], [0, 1.1], [1.2, 0], [1, 1]])
vor = Voronoi(points)
fig = voronoi_plot_2d(vor, show_vertices=False, line_colors='orange',
                line_width=2, line_alpha=0.6, point_size=2)
plt.gca().set_aspect('equal', adjustable='box')
fig.set_size_inches(8,8)
plt.show()

vorにはどんな情報が含まれているか。

What is contained in the variable `vor`?

In [ ]:
vor.__dict__

_pointsが与えた点(4つ)、verticeがvoronoi頂点(外接円の中心)のようだ。よく見ると、辺の情報も含まれている。

 3次元でも4次元でもscipy.spatial.Voronoiを使えばVoronoi分割できるが、残念なことに、既存の関数を使って可視化できるのは2次元まで。

`_points` would correspond to the given points, `vertices` are the voronoi vertices (a voronoi vertex  is the center of a circumlscribed circle.) Edge information is also included.

 別の例。ひまわりの種の配列。

 An another example： Mimic the sequence of sunflower seeds.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

tau  = (5**0.5-1)/2 # Golden ratio
A    = 2*np.pi / tau   # Golden angle

# Place 400 points on a spiral line.
# Place points at a fixed angular interval.
# As the radius increases, the distance between
# two consecutive points increases and the area
# occupied by a single species becomes larger.
# To compensate for this tendency, the radius of
# the helix is chosen to be proportional to the
# square root of the order number of the points.
angles = np.array([A*i for i in range(400)])
radii  = np.array([i**0.5 for i in range(400)])

x = radii * np.cos(angles)
y = radii * np.sin(angles)

plt.scatter(x,y)
plt.axis('square')
plt.show()

In [ ]:
pos = np.vstack([x,y]).T

# https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.voronoi_plot_2d.html
from scipy.spatial import Voronoi, voronoi_plot_2d
vor = Voronoi(pos)
fig = voronoi_plot_2d(vor, show_vertices=False, line_colors='orange',
                line_width=2, line_alpha=0.6, point_size=2)
plt.axis('equal')
plt.xlim(-20,20)
plt.ylim(-20,20)
plt.show()

# Delaunay分割を利用した補間 / Interpolation using Delaunay triangulation

Delaunay分割により三角格子を生成させると、内挿が容易になります。

Interpolation is facilitated by having the Delaunay triangular tessellation.

In [ ]:
from scipy.spatial import Delaunay
import numpy as np

# Positions of the post offices.
postoffices = np.array([[0, -0.1], [0, 1.1], [1.2, 0], [1, 1]])

# Delaunay triangulation
tri = Delaunay(postoffices)
tri


`tri`の中身を見る。

See the content of `tri`.

In [ ]:
tri.__dict__


三角形を描く。

Draw triangles.


In [ ]:
import matplotlib.pyplot as plt
plt.triplot(postoffices[:,0], postoffices[:,1], tri.simplices)
plt.scatter(postoffices[:,0], postoffices[:,1], s=100)
plt.axis('square')
plt.show()


外接円の半径と中心は?

Look for the radii and centers of the circumcircles.


In [ ]:
import numpy as np
def plane(p1,p2,p3):
    """p1,p2,p3を通る平面z=ax+by+cの係数a,b,cを返す

    Return the coefficients a, b, c of the plane z=ax+by+c passing through p1, p2, p3"""
    x1,y1,z1 = p1
    x2,y2,z2 = p2
    x3,y3,z3 = p3
    A = np.array([[x1,y1,1],[x2,y2,1],[x3,y3,1]])
    X = np.array([z1,z2,z3])
    # Aの逆行列とXのドット積
    abc = np.linalg.inv(A) @ X
    return abc

def center_and_radius(a,b,c):
    """平面z=ax+by+cと放物面z=x**2+y**2の交線のxy平面への射影

    Projection of the intersection of the plane z=ax+by+c and the paraboloid z=x**2+y**2 onto the xy-plane"""
    radius = (a**2 + b**2 + 4*c)**0.5/2
    center = (a/2, b/2)
    return center, radius


def circumcircle(p3):
    """
    returns the center and radius of the circumcircle of the given three points.
    """
    a,b,c = plane([p3[0,0], p3[0,1], p3[0,0]**2+p3[0,1]**2],
                  [p3[1,0], p3[1,1], p3[1,0]**2+p3[1,1]**2],
                  [p3[2,0], p3[2,1], p3[2,0]**2+p3[2,1]**2])
    return center_and_radius(a,b,c) # center, radius

# triplet is the labels of vertices of a triangle.
triplet = tri.simplices[0]
center, radius = circumcircle(postoffices[triplet])

center, radius


In [ ]:
import matplotlib.pyplot as plt
plt.triplot(postoffices[:,0], postoffices[:,1], tri.simplices)
plt.scatter(postoffices[:,0], postoffices[:,1], s=100)

circle1 = plt.Circle(center, radius, color='r', fill=False)
plt.scatter(center[0], center[1])
plt.gca().add_patch(circle1)

plt.axis('square')
plt.show()


外接円を全部まとめて描く。

And draw them.


In [ ]:
import matplotlib.pyplot as plt
# Delaunay triangles
plt.triplot(postoffices[:,0], postoffices[:,1], tri.simplices)
plt.scatter(postoffices[:,0], postoffices[:,1], s=100)

# triplet is the labels of vertices of a triangle.
for triplet in tri.simplices:
    center, radius = circumcircle(postoffices[triplet])

    # add a circumcircle
    circle1 = plt.Circle(center, radius, color='r', fill=False)
    plt.gca().add_patch(circle1)
    # center dot
    plt.scatter(center[0], center[1])
    print(center)

plt.axis('square')
plt.show()



てきとうに選んだ3つの家`(0.1, 0.2), (1.5, 0.5), (0.5, 1.05)`が、どちらの郵便局のテリトリー内(三角形の中)にあるかを判定する。

Determine which post office is the nearest from the houses.


In [ ]:
houses = np.array([(0.1, 0.2), (1.5, 0.5), (0.5, 1.05)])



In [ ]:
import matplotlib.pyplot as plt
# Delaunay triangles
plt.triplot(postoffices[:,0], postoffices[:,1], tri.simplices)
plt.scatter(postoffices[:,0], postoffices[:,1], s=100)

# triplet is the labels of vertices of a triangle.
for i, triplet in enumerate(tri.simplices):
    center, radius = circumcircle(postoffices[triplet])

    # add a circumcircle
    circle1 = plt.Circle(center, radius, color='r', fill=False)
    plt.gca().add_patch(circle1)
    # center dot
    plt.scatter(center[0], center[1])
    plt.text(center[0], center[1], f"circ {i}")

# positions of the houses.
plt.scatter(houses[:,0], houses[:,1], s=50)
for i, v in enumerate(houses):
    plt.text(v[0], v[1], f"house {i}")

plt.axis('square')
plt.show()


In [ ]:
tri.find_simplex(houses)

最初と最後の点は三角形1の中にあり、2番目の点はどの三角にも属していない。

三角形1の中で、3つの点がどんな位置にあるか(混合比)を求める。(詳細は理解していません。transformって何?)

`find_simplex()` tells us that the first and last houses are inside the triangle 1 and the second house is outside of all triangles.

Using `transform` in `tri`, we can determine the relative position of a vertex in a triangle 1. (I do not understand the algebra. See the reference.)

In [ ]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.Delaunay.html

# b = tri.transform[1,:2].dot(np.transpose(p - tri.transform[1,2]))
triangle = 1
b = tri.transform[triangle,:2] @  (houses - tri.transform[triangle,2]).T
print(f"Compositions of three vertices of the triangle {triangle}.")
np.c_[b.T, 1 - b.sum(axis=0)]


* 1行目は、混合比が全部0以上1以下、つまり三角形1の内部にある。
* 2行目は負の数や1を越える数があり、三角形1の外の点である。
* 3行目は成分が0の項があるので、点は三角形の辺の上にある。

理解していない手法に頼るのではなく、自力で混合比を求めてみる。



三角形OABの中の点Cの内挿は、$OA=a, OB=b, OC=c,$と書き、$pa+qb=c$となる係数$p,q$を求めることにほかならない。成分で書けば、
$$(p,q)\left(\begin{matrix}a_x, a_y\\ b_x,b_y\end{matrix}\right)=(c_x, c_y)$$
書きかえれば、
$$(p,q)=(c_x, c_y)\left(\begin{matrix}a_x, a_y\\ b_x,b_y\end{matrix}\right)^{-1}$$
これをPython化するとこんな感じになる。

* The first row has mixing ratios all between 0 and 1, i.e., inside triangle 1.
* The second line has a negative number, i.e., the point is outside triangle 1.
* The third line has a term with component zero, so the point is on an edge of triangle 1.

Try to find mixing ratios on our own, rather than relying on methods we do not understand.

Interpolation of point C in triangle OAB is nothing more than writing $OA=a, OB=b, OC=c,$ and finding the coefficients $p,q$ such that $pa+qb=c$. If we write in components,
$$(p,q)\left(\begin{matrix}a_x, a_y\\ b_x,b_y\end{matrix}\right)=(c_x, c_y)$$
Rewriting,
$$(p,q)=(c_x, c_y)\left(\begin{matrix}a_x, a_y\\ b_x,b_y\end{matrix}\right)^{-1}$$


In [ ]:
def interpolate(point, vertices):
    # last point is the origin
    o = vertices[2]
    ab = vertices[:2] - o
    c = point - o
    p, q = c @ np.linalg.inv(ab)
    r = 1 - p - q
    return p, q, r


for house in houses:
    p, q, r = interpolate(house, postoffices[tri.simplices[1]])
    print(p, q, r)


Now we obtain the same result with our own code.

4つの郵便局に、色をあてはめ、それ以外の点はその4色の混合で表す。

Let's assign different colors to the post offices. Any location is expressed by the mixture of the colors.


In [ ]:
colors = np.array([[1,0,0],
                   [0.5,1,0],
                   [0,1,1],
                   [0.5,0,1]])

import matplotlib.pyplot as plt
plt.scatter(postoffices[:,0], postoffices[:,1], c=colors, s=100)

plt.axis('square')
plt.show()


In [ ]:
X = []
Y = []
mixedcolors = []

for ix in range(13):
    for iy in range(11):
        x, y = ix/10, iy/10
        house = np.array([x, y])
        # 点がどっちの三角形に属するか。
        which = tri.find_simplex(house)
        # どの三角にも属しない場合は飛ばす。
        if which < 0:
            continue
        # その三角形を構成する頂点の番号3つを割りだす。
        v3 = tri.simplices[which]
        # 混合比
        mix = interpolate(house, postoffices[v3])
        # その比率で、頂点の色を混ぜる。
        mixedcolor = mix @ colors[v3]
        mixedcolor = np.around(mixedcolor, decimals=4)
        X.append(x)
        Y.append(y)
        mixedcolors.append(mixedcolor)

plt.scatter(X, Y, s=50, facecolor=mixedcolors)
plt.triplot(postoffices[:,0], postoffices[:,1], tri.simplices)
plt.scatter(postoffices[:,0], postoffices[:,1], s=100, facecolor=colors)
plt.show()


## これは何に使えるの? / What is it for?

スライドでも紹介した通り、Voronoi分割のアイディアは、情報を幾何学的に扱ういろんなところに顔を出します。また、2次元の情報の処理のために、一旦三次元に持ちあげて考える、というのも非常に面白い視点だと思います。

* データの補間
* データの分類と最適化
* データの可視化

など。アイディアは4次元以上のデータでも使えるはずで、機械学習の手法としても面白いと思います。

As I mentioned in the slides, the idea of Voronoi tessellation shows up in various places where information is handled geometrically. It is also a very interesting perspective to think about lifting information once into three dimensions in order to process two-dimensional information.

* Data interpolation
* Data classification and optimization
* Data visualization

The idea should also work for data in 4 or more dimensions, and it would be interesting as a machine learning technique.


# 課題 / Practice

岡山県内の郵便局のテリトリーを可視化してみましょう。

Visualize the territories of the post offices in Okayama Prefecture.





## 地図 / Map

行政区域の外形データは、国土地理院が提供しています。

The outline data for administrative areas is provided by the Geospatial Information Authority of Japan (GSI).

In [ ]:
# 国土地理院から地形データ(行政区域の輪郭線)を入手 (この例は岡山県、県コードは33)
# Case of Okayama prefecture (Pref. code 33).
! wget https://nlftp.mlit.go.jp/ksj/gml/data/N03/N03-2021/N03-20210101_33_GML.zip
! unzip N03-20210101_33_GML.zip

In [ ]:
# shapeデータの解析のためのライブラリをcolabにインストール
# Install geopandas to utilize GIS data.
! pip install geopandas

In [ ]:
# shapeデータの読みこみ
import geopandas as gpd
import matplotlib.pyplot as plt

path_shp = "./N03-20210101_33_GML/N03-21_33_210101.shp"
gdf = gpd.read_file(path_shp)
gdf.head()

In [ ]:
f = plt.figure(figsize=(12, 8))
a = f.gca()
for region in gdf.iloc:
    a.plot(*region.geometry.exterior.xy, "k", linewidth=0.5)

## 郵便局の位置 / Locations of the post offices

郵便局の座標情報は国土地理院が提供しています。

Coordinates for the post office are provided by the Geospatial Information Authority of Japan.


https://nlftp.mlit.go.jp/ksj/gml/datalist/KsjTmplt-P30.html

岡山の郵便局の情報は以下にありました。Information on the post offices in Okayama is found here.

https://nlftp.mlit.go.jp//ksj/gml/data/P30/P30-13/P30-13_33.zip



とってきます。

Let's obtain it.

In [ ]:
! wget https://nlftp.mlit.go.jp//ksj/gml/data/P30/P30-13/P30-13_33.zip
! unzip P30-13_33.zip

とりあえず中身を読みます。

and take a look at it.

In [ ]:
# shapeデータの読みこみ
import geopandas as gpd
import matplotlib.pyplot as plt

path_shp = "P30-13_33/P30-13_33.shp"
gdf = gpd.read_file(path_shp)
gdf.head()

プロットしてみます。geopandasのプロットのしかたは知らないのですが、いきあたりばったりでうまく表示できました。

Let's plot the positions.

In [ ]:
ax = gdf.plot("geometry")

Can we overlay the outline on this plot?

In [ ]:
path_shp = "./N03-20210101_33_GML/N03-21_33_210101.shp"
# another geopandas data frame for outline
gdf2 = gpd.read_file(path_shp)

# define a plot area
f = plt.figure(figsize=(12, 8))
a = f.gca()

# draw outlines
for region in gdf2.iloc:
    a.plot(*region.geometry.exterior.xy, "k", linewidth=0.5)

# overlay dots onto the map
gdf.plot("geometry", ax=a)

Can we extract the coordinates of the post offices and apply Voronoi analysis?

In [ ]:
gdf["geometry"]

In [ ]:
gdf["geometry"].__dict__

No clue.

In [ ]:
gdf["geometry"].x

Hit!


In [ ]:
gdf["geometry"].x.to_numpy()

Bingo!

In [ ]:
X = gdf["geometry"].x.to_numpy()
Y = gdf["geometry"].y.to_numpy()

pos = np.array([X,Y]).T
pos

In [ ]:

# https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.voronoi_plot_2d.html
from scipy.spatial import Voronoi, voronoi_plot_2d
vor = Voronoi(pos)
fig = voronoi_plot_2d(vor, show_vertices=False, line_colors='orange',
                line_width=2, line_alpha=0.6, point_size=2)
ax = fig.gca()
ax.set_xlim(133.2,134.5)
ax.set_ylim(34.3,35.4)

# draw outlines
for region in gdf2.iloc:
    ax.plot(*region.geometry.exterior.xy, "k", linewidth=0.5)


In some places near the borders, the post office in a neighboring local government might be closer.